In [ ]:
!pip install llama-index llama-index-llms-groq groq llama-index-embeddings-huggingface ipywidgets


###Without Knowledge Graph

In [ ]:
from IPython.display import display
import ipywidgets as widgets
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    load_index_from_storage
)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
import warnings
import os

warnings.filterwarnings('ignore')

# Set the API key as an environment variable
os.environ["GROQ_API_KEY"] = "ENTER API KEY"

# Now you can access it in your code using os.getenv("GROQ_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# Define your prompt template
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

# Define the context for your prompt
context = "This directory contains multiple documents providing examples and solutions for various programming tasks."

# Data ingestion: load all files from a directory
directory_path = "PATH"  # Update this with your directory path
reader = SimpleDirectoryReader(input_dir=directory_path)
documents = reader.load_data()

# Split the documents into nodes
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

# Set up embedding model and LLM
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

# Create service context
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

# Create and persist the vector store index
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)
vector_index.storage_context.persist(persist_dir="./storage_mini")

# Load the index from storage
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")
index = load_index_from_storage(storage_context, service_context=service_context)

# Create the interactive widgets
input_box = widgets.Text(
    value='Explain Python?',
    placeholder='Type your question here',
    description='Question:',
    disabled=False
)

output_area = widgets.Output()

def on_button_click(b):
    with output_area:
        output_area.clear_output()
        question = input_box.value
        query_prompt = prompt_template.format(context=context, question=question)
        resp = query_engine.query(query_prompt)
        print(resp.response)

button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Ask the question',
    icon='check'
)

button.on_click(on_button_click)

display(input_box, button, output_area)

# Set up query engine
query_engine = index.as_query_engine(service_context=service_context)


**What is Python?**

Python is a high-level, interpreted programming language that provides an extensive standard library and a wide range of facilities for various programming tasks. It offers a simple syntax, making it easy to learn and use.

**Key Features:**

* **Extensive Standard Library**: Python's standard library is very extensive, offering a wide range of facilities for various programming tasks, including file I/O, text processing, data types, numeric and mathematical modules, functional programming modules, and more.
* **Portability**: Python programs are designed to be portable, with modules that abstract away platform-specific details, making it easy to write cross-platform code.
* **Large Community**: Python has an active community, with hundreds of thousands of components available from the Python Package Index.

**Additional Resources:**

* For a more detailed look at Python's standard library, see the library reference manual.
* For an introduction to Python modules, 

### With Knowledge Graph

In [ ]:
!pip install neo4j langchain-experimental

In [ ]:
from IPython.display import display
import ipywidgets as widgets
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext, ServiceContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.llms.groq import Groq
import warnings
import os
from neo4j import GraphDatabase
import spacy

warnings.filterwarnings('ignore')

# ---- NEO4J SETUP ----
neo4j_uri = "ENTER URI"
neo4j_user = "neo4j"
neo4j_password = "ENTER INSTANCE PASSWORD"
driver = GraphDatabase.driver(neo4j_uri, auth=(neo4j_user, neo4j_password))

In [ ]:
# ---- ENVIRONMENT VARIABLES ----
os.environ["GROQ_API_KEY"] = "ENTER API KEY"
GROQ_API_KEY = os.getenv("GROQ_API_KEY")

# ---- PROMPT TEMPLATE ----
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Graph Insights: {graph_insights}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information and graph insights, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

In [ ]:
# Define the context for your prompt
context = "This directory contains multiple documents providing examples and solutions for various programming tasks."

# Data ingestion: load all files from a directory
directory_path = "ENTER PATH"
reader = SimpleDirectoryReader(input_dir=directory_path)
documents = reader.load_data()

# Load spacy model (you can choose a different model)
nlp = spacy.load("en_core_web_sm")

In [ ]:
# Function to extract entities and relationships from documents
def populate_graph(documents, driver, nlp):
    with driver.session() as session:
        for doc in documents:
            doc_text = doc.text  # Assuming each document has a 'text' attribute
            nlp_doc = nlp(doc_text)
            concepts = [ent.text for ent in nlp_doc.ents if ent.label_ == "ORG" or ent.label_ == "PRODUCT"] # Adjust entity types as needed

            for concept in concepts:
                session.run("MERGE (:Concept {name: $concept})", concept=concept)

            for i, concept in enumerate(concepts):
                if i + 1 < len(concepts):
                    next_concept = concepts[i + 1]
                    session.run(
                        """
                        MATCH (c1:Concept {name: $concept}), (c2:Concept {name: $next_concept})
                        MERGE (c1)-[:RELATED_TO]->(c2)
                        """,
                        concept=concept, next_concept=next_concept
                    )

# Populate the Neo4j graph
populate_graph(documents, driver, nlp)

In [ ]:
# Split the documents into nodes
text_splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
nodes = text_splitter.get_nodes_from_documents(documents, show_progress=True)

# Set up embedding model and LLM
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
llm = Groq(model="llama3-70b-8192", api_key=GROQ_API_KEY)

# Create service context
service_context = ServiceContext.from_defaults(embed_model=embed_model, llm=llm)

# Create vector store index
vector_index = VectorStoreIndex.from_documents(documents, show_progress=True, service_context=service_context, node_parser=nodes)
vector_index.storage_context.persist(persist_dir="./storage_mini")

# Load the index from storage
storage_context = StorageContext.from_defaults(persist_dir="./storage_mini")
index = load_index_from_storage(storage_context, service_context=service_context)

In [ ]:
# Create the interactive widgets
input_box = widgets.Text(
    value='Explain Python?',
    placeholder='Type your question here',
    description='Question:',
    disabled=False
)

output_area = widgets.Output()

In [ ]:
#Query Enhancement with Neo4j

def get_graph_insights(question):
  with driver.session() as session:
    result = session.run(
         """
            MATCH (c:Concept)
            WHERE toLower(c.name) CONTAINS toLower($question)
            OPTIONAL MATCH (c)-[r:RELATED_TO]->(other:Concept)
            RETURN c.name AS concept, collect(other.name) AS related_concepts
            """,
         question=question
         )
    insights = []
    for record in result:
       insights.append(f"Concept: {record['concept']}, Related Concepts: {', '.join(record['related_concepts'])}")
       return "\n".join(insights) if insights else "No relevant graph insights found."


In [ ]:
def on_button_click(b):
  with output_area:
    output_area.clear_output()
    question = input_box.value
    graph_insights = get_graph_insights(question)
    query_prompt = prompt_template.format(context=context, graph_insights=graph_insights, question=question)
    resp = query_engine.query(query_prompt)
    print(resp.response)


button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Ask the question',
    icon='check'
)

button.on_click(on_button_click)

display(input_box, button, output_area)

#Query Engine Setup
query_engine = index.as_query_engine(service_context=service_context)

In [16]:
button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Ask the question',
    icon='check'
)

button.on_click(on_button_click)

display(input_box, button, output_area)

# ---- QUERY ENGINE SETUP ----
query_engine = index.as_query_engine(service_context=service_context)


Based on the provided context, I will explain Decimal and its connections.

**Decimal Class**

The Decimal class is a numeric type that provides support for fast correctly rounded decimal floating point arithmetic. It offers several advantages over the float type, including:

* Decimal values can be represented exactly, without rounding errors.
* The Decimal class provides a way to control the rounding mode and precision of arithmetic operations.

**Methods and Functions**

The Decimal class provides several methods and functions to perform various operations, including:

* `number_class(context=None)`: Returns a string describing the class of the operand.
* `quantize(exp, rounding=None, context=None)`: Returns a value equal to the first operand after rounding and having the exponent of the second operand.
* `radix()`: Returns the radix (base) in which the Decimal class does all its arithmetic.
* `remainder_near(other, context=None)`: Returns the remainder from dividing self by other.


In [20]:
button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Ask the question',
    icon='check'
)

button.on_click(on_button_click)

display(input_box, button, output_area)

# ---- QUERY ENGINE SETUP ----
query_engine = index.as_query_engine(service_context=service_context)


**What is an API?**

An API, or Application Programming Interface, is a set of defined rules that enable different applications, services, or systems to communicate with each other. It allows one system to request access to another system's functionality, data, or features, and receive the response in a structured and standardized way.

In the context of the provided documentation, APIs are used to interact with the "email" package, which provides a way to parse, generate, and manipulate email messages. The API allows developers to access the package's functionality, such as parsing email messages, generating MIME documents, and managing MIME content.

Additionally, the documentation mentions other APIs, such as the "http.server" API, which provides a way to handle HTTP requests and responses.

**Additional Helpful Information:**

* APIs can be thought of as a messenger between different systems, allowing them to communicate with each other seamlessly.
* APIs typically define a set of 

In [25]:
button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='',
    tooltip='Ask the question',
    icon='check'
)

button.on_click(on_button_click)

display(input_box, button, output_area)

# ---- QUERY ENGINE SETUP ----
query_engine = index.as_query_engine(service_context=service_context)


**Explanation of Decimal**

The Decimal class is a part of the decimal module in Python, which provides support for fast correctly rounded decimal floating point arithmetic. It offers several advantages over the float type, including:

* Decimal values can exactly represent decimal fractions, unlike floats which are binary fractions.
* The decimal module provides support for fast correctly rounded decimal floating point arithmetic.
* It is particularly useful for financial and monetary calculations where precision and accuracy are crucial.

**Connected Concepts**

The Decimal class is connected to various concepts, including:

* **Rounding**: The Decimal class provides various rounding modes, such as ROUND_HALF_UP, ROUND_HALF_DOWN, ROUND_HALF_EVEN, and more.
* **Quantize**: The quantize method returns a value equal to the first operand after rounding and having the exponent of the second operand.
* **Number Classification**: The number_class method returns a string describing the class